In [1]:
%pylab inline
import geopandas as gpd
from geopandas.tools import sjoin
import pandas as pd
from shapely.geometry import Point,LineString
from IPython.display import display
from pygeocoder import Geocoder
import numpy as np,mapbox as mp
import googlemaps,csv,math,os,re
from collections import Counter
from functools import reduce
from operator import itemgetter
import calendar
from datetime import datetime
import pickle,multiprocessing as mproc, seaborn as sns
from scipy.sparse import csr_matrix,vstack
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

Populating the interactive namespace from numpy and matplotlib


In [2]:
ph_str_gpd = gpd.read_file('../phillydata/Street_Centerline_qgis3.geojson')
# ph_str_bfr_gpd = ph_str_gpd[['STCL2_ID','SEG_ID','geometry']].copy()
# ph_str_bfr_gpd.geometry = ph_str_bfr_gpd.buffer(0.0001)
ph_str_gpd.head().T


,0,1,2,3,4
CLASS,3,3,3,5,4
FNODE_,2,2,1,6,5
LENGTH,449.863,540.083,446.104,447.261,148.216
LPOLY_,0,0,0,0,0
L_F_ADD,1500,400,1600,1600,350
L_HUNDRED,1500,400,1600,1600,300
L_T_ADD,1598,498,1698,1698,398
MULTI_REP,0,0,0,0,0
NEWSEGDATE,None,None,None,None,None
ONEWAY,FT,TF,FT,TF,TF


In [ ]:
purp = pd.read_pickle('Route_Purpose.pkl')
purp['Start'] = pd.DatetimeIndex(purp.Start)
purp['Stop'] = pd.DatetimeIndex(purp.Stop)

o1 = np.loadtxt('Output1.txt',dtype='int')
o2 = open('Output2.txt').readlines()
o2 = np.array([int(re.findall('[0-9]+',t)[0]) for t in o2])
indices = np.sort(np.append(o1,o2))
print("Routes outside "+str(len(o1)))
print("Routes partially outside "+str(len(o2)))
print("Routes completely inside "+str(len(purp)-len(indices)))
new = purp.ix[set(range(len(purp)))-set(indices)]
trip_len = np.array([len(row.geometry.coords) for idx,row in new.iterrows()])
temp = [True if 2<len(row.geometry.coords)<=300 else False for idx,row in new.iterrows()]
new = new.ix[temp].reset_index(drop=True)
print("Routes that are atmost 300 and atleast 3 points long "+str(len(new)))
new.head()
# f = open('Data.pkl','wb')
# pickle.dump(new, f)

In [ ]:
pickle.dump(new,open('Route_Choice.pkl','wb'))

### Constructing Data for Training

In [ ]:
purp = gpd.read_file('tripbytrip_rd2_geojson.geojson')
purp = purp[purp.geometry.type=="LineString"]
purp.reset_index(inplace=True,drop=True)

temp = [True if 2<len(row.geometry.coords)<=300 else False for idx,row in purp.iterrows()]
purp = purp.ix[temp].reset_index(drop=True)
purp = purp.ix[purp[['Purpose','UserId','Start']].drop_duplicates().index].reset_index(drop=True)
temp = purp.geometry.apply(lambda x: x in new.geometry)
purp = purp[temp].reset_index(drop=True)

temp1 = [list(temp.coords) for temp in purp.geometry]
temp2 = [list(temp.coords) for temp in new.geometry]

temp = [temp2.index(t) for t in temp1]

purp['Start'] = pd.to_datetime(purp.Start)
purp['seg'] = temp

purp.to_pickle('Final_Data.pkl')
print(purp.shape)
purp.head().T

In [4]:
purp = pickle.load(open('Final_Data.pkl','rb'))
print(purp.shape)
purp.head().T

(7629, 14)


,0,1,2,3,4
Purpose,Commute,Commute,Shopping,Commute,Commute
Start,2014-09-18 16:54:57,2014-09-19 08:24:54,2014-09-19 16:18:02,2014-09-22 08:12:23,2014-09-22 16:52:26
Stop,2014-09-18T17:21:29,2014-09-19T08:54:03,2014-09-19T16:36:25,2014-09-22T08:34:15,2014-09-22T17:23:57
TripID,10020,10082,10128,10228,10265
UserId,1,1,1,1,1
age,35 - 44,35 - 44,35 - 44,35 - 44,35 - 44
cycling_fr,Daily,Daily,Daily,Daily,Daily
ethnicity,White,White,White,White,White
gender,Male,Male,Male,Male,Male
geometry,LINESTRING (-75.18906357192628 39.957607633485...,LINESTRING (-75.14535405814325 39.963978383195...,LINESTRING (-75.19162367019618 39.955083562056...,LINESTRING (-75.14779781921396 39.963528080956...,LINESTRING (-75.18931917633887 39.956421811717...


In [ ]:
# coding=utf-8
import geopandas as gp
from shapely.geometry import Point, LineString, Polygon

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    from math import radians, cos, sin, asin, sqrt
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    km = 6367 * c
    m = km * 1000
    return m


def ptfromln(pt, ln):
    """
    project pt to ln, compute haversine distance between projected pt and pt
    :param pt: shapely.geometry.Point, (lon, lat) point
    :param ln: shapely.geometry.LineString, [(lon,lat)] points
    :return: distance in meters
    """
    n_pt = ln.interpolate(ln.project(pt))
    lon1, lat1 = n_pt.coords[0]
    lon2, lat2 = pt.coords[0]
    return haversine(lon1, lat1, lon2, lat2)

def crs_prepossess(gpdf, init_crs, bfr_crs):
    """
    create a shallow copy of gpdf; check the init crs of gpdf, if None, assign init_crs; change crs of copy to bfr_crs
    :param gpdf: geopandas.GeoDataFrame
    :param init_crs: init_crs epsg code
    :param bfr_crs: target crs epsg code used for buffering
    :return: a shallow copy of gpdf in bfr_crs
    """
    gpdf_crs = gpdf.copy()
    if gpdf_crs.crs == None:
        gpdf_crs.crs = {'init': u'epsg:{}'.format(init_crs)}
    return gpdf_crs.to_crs(epsg=bfr_crs)


# ########## functions assigning ln(segment) to objs #############
def pts2segs(pts, lns, bfr_crs, init_crs=4326, close_jn_dist=10, far_jn_dist=20):
    """
    1. close jn: buffer pts in bfr_crs with close_jn_dist, use sjoin to find segment(s) intersected with buffered pts
    2. far jn: for pts without any segment in close jn, buffer them with far_jn_dist and find nearest segment
    :param pts: geopandas.GeoDataFrame
    :param lns: geopandas.GeoDataFrame
    :param bfr_crs: target crs epsg code used for buffering
    :param init_crs: init_crs epsg code, default 4326(lat lon)
    :param close_jn_dist: close join distance, allowing multiple segments for one point(assumed as intersection)
    :param far_jn_dist: far join distance, find the nearest segment for one point
    :return: pandas.DataFrame, columns=[pt_index, ln_index]
    """

    import pandas as pd
    index_pt, index_ln = 'index_pt', 'index_ln'

    lns_crs = crs_prepossess(lns, init_crs, bfr_crs)
    pts_crs = crs_prepossess(pts, init_crs, bfr_crs)

    close_jn = pts_crs.copy()
    close_jn.geometry = close_jn.buffer(close_jn_dist)
    close_jn = gp.tools.sjoin(close_jn, lns_crs)[['index_right']]

    close_jn_pts = set(pd.unique(close_jn.index))
    far_jn = pts_crs[~pts_crs.index.isin(close_jn_pts)].copy()
    
    if not far_jn.empty:
        far_jn.geometry = far_jn.buffer(far_jn_dist)
        far_jn = gp.tools.sjoin(far_jn, lns_crs)[['index_right']]
        # calculate haversine distance
        far_jn = pd.merge(lns[['geometry']], far_jn, left_index=True, right_on=['index_right'])
        far_jn = pd.merge(pts[['geometry']], far_jn, left_index=True, right_index=True)
        far_jn['dis'] = far_jn.apply(lambda x: ptfromln(x.geometry_x, x.geometry_y), axis=1)
        # keep ln with minimum distance to pt
        far_jn = far_jn.groupby(level=0).apply(lambda x: x.iloc[x.dis.values.argmin()][['index_right']])
        pts_has_ln = close_jn.append(far_jn).reset_index()
    else:
        pts_has_ln = close_jn.reset_index()

    pts_has_ln.columns = [index_pt, index_ln]

    pts_no_ln = pts[~pts.index.isin(pd.unique(pts_has_ln[index_pt]))].copy()
    return pts_has_ln, pts_no_ln


if __name__ == '__main__':
    import geopandas as gp
    # points are (lon, lat)
    from shapely.geometry import Point
#     idx = 216
#     lonlats = list(new.geometry[idx].coords)
# #     lonlats = snap(new.iloc[idx])
#     pts = [Point(lonlat) for lonlat in lonlats]
#     pts_gpdf = gp.GeoDataFrame(pts, columns=['geometry'])
#     # get segment index of segs for each point
#     pts_segs_idx, pts_no_segs = pts2segs(pts_gpdf, ph_str_gpd, bfr_crs=3559, init_crs=4326, close_jn_dist=10, far_jn_dist=30)
#     # get the STREETSEGID for each point
#     pts_segids = pts_segs_idx.merge(ph_str_gpd[['SEG_ID']], left_on=['index_ln'], right_index=True)
#     pts_segids.sort_values(by=['index_pt'],inplace=True)
#     pts_segids.reset_index(inplace=True,drop=True)
#     map_plot(idx,True,list(set(pts_segids.index_ln)),True)

In [ ]:
map_plot(idx,False,list(set(pts_segids.index_ln)),True)

In [ ]:
pts_segids

### POI imputation of missing data

In [ ]:
for idx in upoi[t==1].index:
    lonlats = list(new.geometry[idx].coords)
    #     lonlats = snap(new.iloc[idx])
    pts = [Point(lonlat) for lonlat in lonlats]
    pts_gpdf = gpd.GeoDataFrame(pts, columns=['geometry'])
    # get segment index of segs for each point
    pts_segs_idx, pts_no_segs = pts2segs(pts_gpdf, ph_str_gpd, bfr_crs=3559, init_crs=4326, close_jn_dist=10, far_jn_dist=30)
    # get the STREETSEGID for each point
    pts_segids = pts_segs_idx.merge(ph_str_gpd[['SEG_ID']], left_on=['index_ln'], right_index=True)
    pts_segids.sort_values(by=['index_pt'],inplace=True)
    pts_segids.reset_index(inplace=True,drop=True)
    temp1 = pts_segids['index_ln'].groupby(pts_segids['index_pt']).apply(list)
    temp = [set(u)&set(v) for u,v in list(zip(temp1,temp1[1:]))]
    if(not upoi.start[idx]):
        ans = temp[0]
        i = 1
        while(not ans):
            ans = temp[i]
            i = i+1
        upoi.start[idx] = ans
    else:
        ans = temp[-1]
        i = len(temp)-2
        while(not ans):
            ans = temp[i]
            i = i - 1
        upoi.end[idx] = ans

In [ ]:
temp1 = pts_segids['index_ln'].groupby(pts_segids['index_pt']).apply(list)
temp = [set(u)&set(v) for u,v in list(zip(temp1,temp1[1:]))]

for i,val in enumerate(temp):
    if(len(val)>1): 
        left = i-1
        right = i+1
        if(left<0): left = 0
        if(right==len(temp)): right = len(temp)-1
        if(not t1):
            t2 = temp[right] & val
            if(not t2):
                print(i)
            else:
                temp[i] = t2
        else: temp[i] = t1 
temp
# map_plot(idx,False,list(set.union(*temp)),True)

In [ ]:
len(segments.segments[146])

In [ ]:
map_plot(idx,True,list(set(flat(temp1[6:8]))),False)

### Break

In [ ]:
row = new.ix[2279]
pts = list(row.geometry.coords)
samay = [(row.Start + k*(row.Stop-row.Start)/(len(pts)-1)).strftime('%H:%M:%S') for k in range(len(pts))]

rang = list(zip(*pts))
rang = [list(temp) for temp in rang]
df = list(zip(samay,rang[0],rang[1]))
data = pd.DataFrame(df,columns=['time','longitude','latitude'])
data = data[6:8]
service = mp.MapMatcher()
gps = snap_to_road(service,data)

gps

In [ ]:
total = set.union(*segments.segments)

In [ ]:
segments.segments

In [ ]:
upoi[t==1]

In [ ]:
file_writer(lonlats,'Trip1.csv')

In [ ]:
list(new.geometry[1768].coords)

In [ ]:
new.geometry[1768]

In [ ]:
segments.segments[1768]

In [ ]:
lonlats1 = list(new.geometry[430].coords)
file_writer(lonlats1,'Trip1.csv')
lonlats2 = snap(new.iloc[430])
file_writer(lonlats2,'Trip2.csv')

In [ ]:
import datetime
import mapbox as mp
from snap_to_road import *

def snap(row):

    pts = list(row.geometry.coords)
    samay = [(row.Start + k*(row.Stop-row.Start)/(len(pts)-1)).strftime('%H:%M:%S') for k in range(len(pts))]

    rang = list(zip(*pts))
    rang = [list(temp) for temp in rang]
    df = list(zip(samay,rang[0],rang[1]))
    data = pd.DataFrame(df,columns=['time','longitude','latitude'])

    
    service = mp.MapMatcher()
    gps = snap_to_road(service,data)

    return gps

In [ ]:
total = list()
new = pd.read_pickle('Data.pkl')
for idx,row in new.iterrows():
    if(idx==5): break
    lonlats = snap(row)
    pts = [Point(lonlat) for lonlat in lonlats]
    pts_gpdf = gpd.GeoDataFrame(pts, columns=['geometry'])

    # get segment index of segs for each point
    pts_segs_idx, pts_no_segs = pts2segs(pts_gpdf, ph_str_gpd, bfr_crs=3559, init_crs=4326, close_jn_dist=10, far_jn_dist=30)

    # get the STREETSEGID for each point
    pts_segids = pts_segs_idx.merge(ph_str_gpd[['SEG_ID']], left_on=['index_ln'], right_index=True)
    pts_segids.sort_values(by=['index_pt'],inplace=True)
    pts_segids.reset_index(inplace=True,drop=True)

    temp = pts_segids['index_ln'].groupby(pts_segids['index_pt']).apply(list)
    final = set([s[0] for s in temp if len(s)==1])
    temp = [set(u)&set(v) for u,v in list(zip(temp,temp[1:]))]
    final = final | set([list(s)[0] for s in temp if len(s)==1])
    for idx,val in enumerate(temp):
        if(len(val)>1 and len(val&final)==0): 
            final = final | val
    total.append([idx,final])
    

### Multiprocessing

In [ ]:
import multiprocessing as mproc

if __name__=='__main__':
    
    def apply_fn(row):
        
        print(row.name)
        lonlats = snap(row)
        pts = [Point(lonlat) for lonlat in lonlats]
        pts_gpdf = gpd.GeoDataFrame(pts, columns=['geometry'])

        pts_segs_idx, pts_no_segs = pts2segs(pts_gpdf, ph_str_gpd, bfr_crs=3559, init_crs=4326, close_jn_dist=10, far_jn_dist=30)

        pts_segids = pts_segs_idx.merge(ph_str_gpd[['SEG_ID']], left_on=['index_ln'], right_index=True)
        pts_segids.sort_values(by=['index_pt'],inplace=True)
        pts_segids.reset_index(inplace=True,drop=True)

        temp = pts_segids['index_ln'].groupby(pts_segids['index_pt']).apply(list)
        final = set([s[0] for s in temp if len(s)==1])
        temp = [set(u)&set(v) for u,v in list(zip(temp,temp[1:]))]
        final = final | set([list(s)[0] for s in temp if len(s)==1])
        for idx,val in enumerate(temp):
            if(len(val)>1 and len(val&final)==0): 
                final = final | val
        return pd.Series({'segments' : final})

    def call_apply_fn(df):
        return df.apply(apply_fn,axis=1)

    new = pd.read_pickle('Data.pkl')
    pool = mproc.Pool(mproc.cpu_count()) 
    results = pool.map(call_apply_fn, np.array_split(new,mproc.cpu_count()))
    temp = pd.concat(results)
    temp.to_pickle('Segments.pkl')
    

In [ ]:
np.argmax(np.array([len(t.coords) for t in new.geometry]))

In [ ]:
from snap_to_road import *
row = new.ix[3899]
pts = list(row.geometry.coords)
samay = [(row.Start + k*(row.Stop-row.Start)/(len(pts)-1)).strftime('%H:%M:%S') for k in range(len(pts))]

rang = list(zip(*pts))
rang = [list(temp) for temp in rang]
df = list(zip(samay,rang[0],rang[1]))
df = pd.DataFrame(df,columns=['time','longitude','latitude'])
service = mp.MapMatcher()
print(len(pts))
snap_to_road(service,df)


In [ ]:
from multiprocessing import Pool

def apply_fn(row):
    lonlats = list(row['geometry'].coords)
    pts = [Point(lonlat) for lonlat in lonlats]
    pts_gpdf = gpd.GeoDataFrame(pts, columns=['geometry'])
    try:
        pts_segs_idx, pts_no_segs = pts2segs(pts_gpdf, ph_str_gpd, bfr_crs=3559, init_crs=4326, close_jn_dist=10, far_jn_dist=30)
    except ValueError:
        return 0
    if(len(pts_no_segs)>0):
        return 1
    return 2

def call_apply_fn(df):
    return df['geometry'].apply(apply_fn,axis=1)

pool = Pool(4)
pool_results = pool.map(call_apply_fn, np.array_split(purp.ix[0:9],mproc.cpu_count()))
pool.close() 
pool.join() 
total = pd.concat(pool_results)
total.to_pickle('Indices.pkl')

In [ ]:
with open('Output.txt','w') as f:
    for item in pool_results:
        f.write("%s\n" % item)

In [ ]:
def flat(l):
    return [item for sublist in l for item in sublist]

In [ ]:
traces = pd.read_pickle('Segments.pkl')
traces.head().T

In [ ]:
t = list(segments.segments[idx])
t.append(35943)

In [ ]:
def map_plot(idx,flag1,seg,flag2):
    if(flag1): lonlats = snap(new.iloc[idx])
    else: lonlats = list(new.geometry[idx].coords)
    pts = [Point(lonlat) for lonlat in lonlats]
    start = 0
    end = len(pts)
    t = list(segments.segments[idx])
    if(flag2): inpt = ph_str_gpd.iloc[t]
    else: inpt = ph_str_gpd.iloc[seg]
    otpt = pts[start:end+1]
    gpdfs = [inpt, gpd.GeoDataFrame(otpt,columns=['geometry'])]
    create_map_visualization(html_title, file_path, file_name, lat, lon, zoom, init_layers, map_layers, binding_data, gpdfs)

In [ ]:
lonlats[-1]

In [ ]:
temp = []
for idx, row in purp.ix[0:4].iterrows():
    lonlats = list(row['geometry'].coords)
    pts = [Point(lonlat) for lonlat in lonlats]
    pts_gpdf = gp.GeoDataFrame(pts, columns=['geometry'])
    try:
        pts_segs_idx, pts_no_segs = pts2segs(pts_gpdf, ph_str_gpd, bfr_crs=3559, init_crs=4326, close_jn_dist=10, far_jn_dist=30)
    except ValueError:
        temp.append(0)
        continue
    if(len(pts_no_segs)>0):
        temp.append(1)
    else: temp.append(2)

temp

In [ ]:
lonlats = list(purp[purp.TripID=='1377'].geometry.iloc[0].coords)
pts = [Point(lonlat) for lonlat in lonlats]
pts_gpdf = gp.GeoDataFrame(pts, columns=['geometry'])
try:
    pts_segs_idx, pts_no_segs = pts2segs(pts_gpdf, ph_str_gpd, bfr_crs=3559, init_crs=4326, close_jn_dist=10, far_jn_dist=30)
except ValueError:
    print('HiHi')
if(len(pts_no_segs)>=1):
    print('HI')


In [ ]:
file_writer(lonlats,'trip.csv')

In [ ]:
pts_segids[(pts_segids['index_pt']>=11)&(pts_segids['index_pt']<=15)]

In [ ]:
pt_idx = list(range(150,160))
for ptx in pt_idx:
    pt = pts_gpdf.geometry[ptx]

    # the segments that are near pt 45
    seg_indices = pts_segids[(pts_segids['index_pt']==ptx)].index_ln
    print("Point Index: "+str(ptx))
    temp = ph_str_gpd.loc[seg_indices]['geometry'].index
    temp2 = ph_str_gpd.loc[seg_indices]['geometry'].apply(lambda x: ptfromln(pt, x))
    temp3 = ph_str_gpd.loc[seg_indices]['SEG_ID']
    temp = sorted(list(zip(temp,temp3,temp2)),key=itemgetter(1))
    for val in temp:
        print(val)

In [ ]:
pt_idx = 15
pt = pts_gpdf.geometry[pt_idx]

# the segments that are near pt 45
seg_indices = list(pts_segids[(pts_segids['index_pt']==pt_idx)].index_ln)
seg_indices
temp = ph_str_gpd.loc[seg_indices]['geometry'].index
temp2 = ph_str_gpd.loc[seg_indices]['geometry'].apply(lambda x: ptfromln(pt, x))
sorted(list(zip(temp,temp2)),key=itemgetter(1))

In [ ]:
# the pt with index 45
pt_idx = [44,45,46]
for ptx in pt_idx:
    pt = pts_gpdf.geometry[ptx]

    # the segments that are near pt 45
    seg_indices = pts_segids[(pts_segids['index_pt']==ptx)].index_ln
    print("Point Index: "+str(ptx))
    temp = ph_str_gpd.loc[seg_indices]['geometry'].index
    temp2 = ph_str_gpd.loc[seg_indices]['geometry'].apply(lambda x: ptfromln(pt, x))
    temp3 = ph_str_gpd.loc[seg_indices]['SEG_ID']
    temp = sorted(list(zip(temp,temp3,temp2)),key=itemgetter(1))
    for val in temp:
        print(val)


In [ ]:
start = 0
end = len(pts)
temp = pts_segids[(pts_segids['index_pt']>=start)&(pts_segids['index_pt']<=end)].index_ln
# inpt = ph_str_gpd.iloc[temp]
inpt = ph_str_gpd.iloc[final]
otpt = pts[start:end+1]
gpdfs = [inpt, gpd.GeoDataFrame(otpt,columns=['geometry'])]
# gpdfs = [ph_str_gpd, gpd.GeoDataFrame(pts,columns=['geometry'])]
create_map_visualization(html_title, file_path, file_name, lat, lon, zoom, init_layers, map_layers, binding_data, gpdfs)

In [ ]:
from leaflet_creation import create_map_visualization
html_title = 'openstreetmap elements'
file_path = ''
file_name = 'test creation of leaflet'
lon, lat = -75.17445310502671, 39.964070156312744  #D.C.
zoom = 12
init_layers = ['streets', 'stsg']
map_layers = ['light','streets', 'satellite']
binding_data=[['stsg','street segment'],['stsg1','street segment1']]
ph_str_gpd['color'] = '#aa0'
# gpdf2['color'] = '#0a0'
# inpt = 
# otpt = pts[45:48]
gpdfs = [ph_str_gpd, gpd.GeoDataFrame(pts,columns=['geometry'])]
create_map_visualization(html_title, file_path, file_name, lat, lon, zoom, init_layers, map_layers, binding_data, gpdfs)

In [ ]:
def file_writer(new,filename):
    with open(filename,'w') as f:
        writer = csv.writer(f)
        writer.writerow(["Longitude","Latitude"])
        writer.writerows(new)

In [ ]:
crime = pd.read_csv('../phillydata/Crime_Details.csv')
park = pd.read_csv('../phillydata/Philly_Parking_Violation_Details_2014-2015.csv')
p311 = pd.read_csv('../phillydata/Philly_311_Details_2015-16.csv')
cols = pd.read_csv('../phillydata/Philly_Collision_Details_2013-14.csv')
bike = pd.read_csv('../phillydata/Philly_Bikelanes.csv')
slope = pd.read_csv('../phillydata/Slope_Segment.csv')

temp = crime.groupby(['STREETSEGID']).sum().reset_index()
feat = temp[['STREETSEGID','CRIME_COUNT']]
feat.columns = ['SEG_ID','CRIME']

temp = park.groupby(['STREETSEGID']).sum().reset_index()
feat2 = temp[['STREETSEGID','TYPE_COUNT']]
feat2.columns = ['SEG_ID','PARKING']

temp = p311.groupby(['SEG_ID']).sum().reset_index()
feat3 = temp[['SEG_ID','monthly_311_request_count']]
feat3.columns = ['SEG_ID','311_REQUEST']

temp = cols.groupby(['SEG_ID']).sum().reset_index()
feat4 = temp[['SEG_ID','TOTAL_COLLISION_COUNT']]
feat4.columns = ['SEG_ID','COLLISION']

slope.reset_index()

feat5 = ph_str_gpd[['SEG_ID','SHAPE_LEN']]

dfs = [feat,feat2,feat3,feat4,bike,slope,feat5]
feat_final = reduce(lambda left,right: pd.merge(left,right,on='SEG_ID',how='outer'), dfs)
feat_final.fillna(0,inplace=True)
feat_final.head()

In [ ]:
elev = pd.read_csv('../phillydata/Philly_Elevation_SegId.csv')


In [ ]:
elev = pd.read_csv('../phillydata/Philly_Elevation_SegId.csv')
seg_id = sort(list(set(elev['SEG_ID'])))
slope = []
for seg in seg_id:
    temp2 = elev[elev['SEG_ID']==seg].Elevation
    temp3 = max(temp2)-min(temp2)
    if argmax(temp2)<argmin(temp2):
        temp3 = -temp3
    slope.append(temp3)
slope

In [ ]:
feat6 = pd.DataFrame(list(zip(seg_id,slope)))
feat6.columns = ['SEG_ID','ELEV_SLOPE']
feat6.to_csv("../phillydata/Slope_Segment.csv")

In [ ]:
buffer = set()
final = set()
temp = []
for i in range(1,len(new)):
    s1 = set(pts_segids[pts_segids['index_pt']==i-1].index_ln)
    s2 = set(pts_segids[pts_segids['index_pt']==i].index_ln)
    t = s1 & s2
    temp.append(list(t))
    if(len(t)==0):
        continue
    elif(len(t)==1):
        final = final | t
    else:
        buffer = buffer | t
        
    if buffer:
        tmp = buffer & t
        if(len(tmp)==1):
            final = final | tmp
            buffer = set()
#         else: print(i,len(tmp))
#     if not t:
#         t1 = ','.join(str(v) for v in s1)
#         t2 = ','.join(str(v) for v in s2)
#         print(str(i)+" "+t1+"----"+t2)

In [ ]:
pts_segids[(pts_segids['index_pt']>=115)&(pts_segids['index_pt']<=128)]

In [ ]:
temp

In [ ]:
unq_set = set()
bfr = set()
for i in range(0,50):#len(new)):
    temp = set(pts_segids[pts_segids['index_pt']==i].index_ln)
    t1 = ','.join(str(v) for v in temp)
    t2 = ','.join(str(v) for v in bfr)
    t3 = ','.join(str(v) for v in unq_set)
    print(str(i)+" Current: "+t1+" Buffer: "+t2+" Total: "+t3)
    if not bfr:
        if not len(temp & unq_set):
            bfr = bfr | temp
        continue
    if len(temp)==1:
        unq_set = unq_set | temp
        val = list(temp)[0]
        if val in bfr:
            bfr = set()
            continue
        else: 
            print(i)
    bfr = bfr & temp
    if not bfr:
        print("P1 "+str(i))
    if len(bfr)==1:
        val = list(bfr)[0]
        if not val in unq_set:
            print("P2")
            unq_set = unq_set | bfr
        bfr = set()

In [ ]:
for i in range(0,len(new)):
    if not (unq_set & set(pts_segids[pts_segids['index_pt']==i].index_ln)):
        print(i)

In [ ]:
pts_segids[(pts_segids['index_pt']>=13)&(pts_segids['index_pt']<=29)]

In [ ]:
x = Counter(pts_segids.index_ln)
sorted(x.items(), key=lambda pair: pair[1], reverse=True)

### New Data

In [ ]:
trip = gpd.read_file('cyclephilly_rd2_geojson/cyclephilly_rd2_geojson.geojson')
trip.head().T

In [ ]:
mpta = gpd.read_file('trip_detail_rd2_shp/trip_detail_rd2.shp')

In [ ]:
mota.head().T

In [ ]:
len(temp)

In [ ]:
Counter(temp.Purpose)

In [ ]:
Counter(temp.cycling_fr)

In [ ]:
trip = gpd.read_file('nodes_geojson/CyclePhilly_Network_nodes.geojson')
print(trip.shape)
trip.head().T

In [ ]:
trip2 = gpd.read_file('nodes_shp/CyclePhilly_Network_nodes.shp')

In [ ]:
sum(trip.NO.isin([823213]))

In [ ]:
hist = np.array([len(t.coords) for t in temp.geometry])
print(len(hist[hist>30]))
print(mean(hist))

In [ ]:
print(len(hist[hist>20]))

In [ ]:
Counter(hist)

In [ ]:
hist2 = np.array([len(t.coords) for t in purp.geometry if t.type!="MultiLineString"])
print(len(hist2[hist2>30]))
print(median(hist2))

In [ ]:
len(temp)

In [ ]:
catg = list(sorted(set(purp.Purpose)))
total_set = []
for cat in catg:
    subst = purp[purp.Purpose==cat]
    temp = [list(temp.coords) for temp in subst.geometry]
    temp = list(set(map(tuple, temp)))
    temp = list(map(LineString,temp)) #subst[subst.geom_equals(t)].index[0]
    unique_set = []
    for t in temp:
        for idx,tr in subst.iterrows():
            if tr.geometry==t: 
                unique_set.append(idx)
                break 
    total_set.extend(unique_set)
    print(cat,len(set(total_set)),len(total_set))

In [ ]:
filtered_purp = purp.ix[total_set]

In [ ]:
filtered_purp.to_pickle('Route_Purpose.pkl')

In [ ]:
purp = pd.read_pickle('Route_Purpose.pkl')
print(purp.shape)
purp.head().T

In [ ]:
purp[purp.geom_equals(purp.geometry[515])]

In [ ]:
temp = purp.geometry[0:20]

In [ ]:
start = [datetime.strptime(temp, '%Y-%m-%dT%H:%M:%S')for temp in purp.Start]
date = [temp.strftime("%Y/%m/%d") for temp in start]

In [ ]:
start = pd.DatetimeIndex(purp.Start)
stop = pd.DatetimeIndex(purp.Stop)
start = []

In [ ]:
purp['date'] = [temp.date() for temp in purp.Start]
purp['month'] = [temp.month for temp in purp.Start]
purp['hour'] = [temp.hour for temp in purp.Start]
mon = list(range(1,13))
temp = purp[purp.UserId!='4']
count = [len(set(temp[temp.month==mn].UserId)) for mn in mon]

rect = plt.bar(mon,count,bar_width)
const = 20
plt.title('Bikers/Month',fontsize=16)
plt.ylabel('Number of Unique Bikers',fontsize=12)
plt.xlabel('Month',fontsize=12)
plt.yticks(np.arange(0,max(count)+2*const,const))
plt.xticks(index + bar_width / 2, names)
plt.xlim([0,14])
autolabel(rect)
savefig('Unique_Users_by_Month.jpg')
# date = sorted(set(purp.date))
# temp = purp[purp.UserId!='4']
# count = [len(set(temp[temp.date==dt].UserId)) for dt in date]

# temp = list(zip(*daily))
# daily = temp[0]
# count = temp[1]
# list(zip(daily,count))

In [ ]:
bike_month = count

In [ ]:
names = list(range(24))
temp = purp[purp.UserId!='4']
count = [len(set(temp[temp.hour==nm].UserId)) for nm in names]

index = np.arange(0,48,2)
bar_width = 1
fig,ax = plt.subplots()
rect = ax.bar(index,count,bar_width)
const = 20

plt.title('Bikers/Hours',fontsize=16)
plt.ylabel('Number of Unique Bikers',fontsize=12)
plt.xlabel('Hours',fontsize=12)
plt.yticks(np.arange(0,max(count)+2*const,const))
ax.set_xticks(index+bar_width)
ax.set_xlim(-2, 50)
plt.xticks(index + bar_width / 2, names)
autolabel(rect)
savefig('Unique_Users_by_Hours.jpg')

In [ ]:
def autolabel(rects):
    """
    Attach a text label above each bar displaying its height
    """
    for rect in rects:
        height = rect.get_height()
        plt.text(rect.get_x() + rect.get_width()/2., height,
                '%d' % int(height),
                ha='center', va='bottom')
        
month = [temp.month for temp in purp.Start]
names = list(calendar.month_abbr)[1:]
data = Counter(month)
data = list(data.items())
temp = list(zip(*data))
mon = temp[0]
count = temp[1]
index = np.arange(12)
bar_width = 0.9
rect = plt.bar(index,count,bar_width)
const = 200
plt.title('Trips/Month',fontsize=16)
plt.ylabel('Number of Trips',fontsize=12)
plt.xlabel('Month',fontsize=12)
plt.yticks(np.arange(0,max(count)+2*const,const))
plt.xticks(index + bar_width / 2, names)
plt.xlim([-1,13])
autolabel(rect)
# plt.show()
savefig('Total_Trips_by_Month.jpg')

In [ ]:
trip_month = count

In [ ]:
count = np.array(trip_month)/np.array(bike_month)
index = np.arange(12)
bar_width = 0.9
rect = plt.bar(index,count,bar_width)
const = 2
plt.title('Normalized_Trips/(Users,Month)',fontsize=16)
plt.ylabel('Number of Trips',fontsize=12)
plt.xlabel('Month',fontsize=12)
plt.yticks(np.arange(0,max(count)+2*const,const))
plt.xticks(index + bar_width / 2, names)
plt.xlim([-1,13])
autolabel(rect)
savefig('Normalized_Trips_Users_Month.jpg')

In [ ]:
names = list(calendar.day_abbr)
day = [names[temp.dayofweek] for temp in purp.Start]
data = Counter(day)
count = [data[t] for t in names]

index = np.arange(7)
bar_width = 0.9
rect = plt.bar(index,count,bar_width)
const = 200
plt.title('Trips/Day',fontsize=16)
plt.ylabel('Number of Trips',fontsize=12)
plt.xlabel('Day',fontsize=12)
plt.yticks(np.arange(0,max(count)+2*const,const))
plt.xticks(index + bar_width / 2, names)
autolabel(rect)
plt.xlim([-1,8])
# plt.show()
savefig('Total_Trips_by_Day.jpg')

In [ ]:
hour = [temp.hour for temp in purp.Start]
data = Counter(hour)
data = list(data.items())
temp = list(zip(*data))
mon = temp[0]
count = temp[1]
names = list(range(24))
index = np.arange(0,48,2)
bar_width = 1
fig,ax = plt.subplots()
rect = ax.bar(index,count,bar_width)
const = 200

plt.title('Trips/Hours',fontsize=16)
plt.ylabel('Number of Trips',fontsize=12)
plt.xlabel('Hours',fontsize=12)
plt.yticks(np.arange(0,max(count)+2*const,const))
ax.set_xticks(index+bar_width)
ax.set_xlim(-2, 50)
plt.xticks(index + bar_width / 2, names)
autolabel(rect)
# plt.show()
savefig('Total_Trips_by_hours.jpg')

In [ ]:
counts = purp.groupby(['age','gender','ethnicity','income']).size()
counts

In [ ]:
temp = list(counts.index)
def f(x):
    if not re.findall('no',x):
        return True
    return False
temp = [t for t in temp if False not in list(map(f,t))]

In [ ]:
def life_saver(l):
    print("\n".join([str(s) for s in l]))

In [ ]:
temp = gpd.read_file('cyclephilly_rd2_geojson/cyclephilly_rd2_geojson.geojson')
temp.head().T

In [ ]:
def plot_bar(names,count,x,y):
    index = np.arange(len(names))
    bar_width = 0.9
    plt.figure(1)
    if(x=='Trips'):
        no = 121        
    else:
        no = 122
    
    plt.subplot(no)
    rect1 = plt.bar(index,count,bar_width)
    const = int(max(count)*0.25)
    plt.title(x+'/'+y,fontsize=14)
    plt.ylabel('# of '+x,fontsize=12)
    plt.xlabel(y,fontsize=12)
    plt.yticks(np.arange(0,max(count)+2*const,const))
    plt.xticks(index + bar_width / 2, names,rotation = 'vertical')
    autolabel(rect1)
    plt.xlim([-1,len(names)+1]);
    plt.tight_layout()
#     plt.setp(names)
#     if(no==211): plt.show()
    if(no==122):
        plt.savefig(y+'.jpg')
    

name = 'ethnicity'
data1 = Counter(purp[purp[name]!='no data'][name])
names = list(data1.keys())
count = list(data1.values())
if(name=='income'):
    names = [re.sub('999','',t) for t in names]
    names = [re.sub('000','',t) for t in names]
    names = [re.sub(',','',t) for t in names]
    names[2],names[4] = names[4],names[2]
    count[2],count[4] = count[4],count[2]
elif(name=='age' or name=='ethnicity'):
    t = list(zip(*sorted(data1.items(),key=lambda k: k[0])))
    names = t[0]
    count = t[1]

plot_bar(names,count,'Trips',name.title())

counts = purp.groupby(['UserId','age','gender','ethnicity','income']).size().reset_index()
data2 = Counter(counts[counts[name]!='no data'][name])
count = list(data2.values())
if(name=='gender'):
    names = list(data2.keys())
elif(name=='ethnicity' or name=='age' ):
    t = list(zip(*sorted(data2.items(),key=lambda k: k[0])))
    names = t[0]
    count = t[1]
plot_bar(names,count,'Users',name.title())


In [ ]:
segments = pd.read_pickle('Segments.pkl')
len(segments)

In [ ]:
len(unique([item for l in unique(segments.segments) for item in l]))

### Yearly Count

In [ ]:
Counter([t.year for t in new.Start])

### POI's

In [ ]:
poi = pd.read_csv('feature_poi_ph.csv',index_col=0)
poi.head().T

In [ ]:
segments = pd.read_pickle('Segments.pkl')

In [ ]:
def apply_fn(row):
    t = list(row.coords)
    pts = [Point(lonlat) for lonlat in [t[0],t[1],t[-2],t[-1]]]
    pts_gpdf = gpd.GeoDataFrame(pts, columns=['geometry'])
    pts_segs_idx, pts_no_segs = pts2segs(pts_gpdf, ph_str_gpd, bfr_crs=3559, init_crs=4326, close_jn_dist=10, far_jn_dist=30)

    pts_segids = pts_segs_idx.merge(ph_str_gpd[['SEG_ID']], left_on=['index_ln'], right_index=True)
    pts_segids.sort_values(by=['index_pt'],inplace=True)
    pts_segids.reset_index(inplace=True,drop=True)
    pts_segids.T

    t1 = set(pts_segids.index_ln[pts_segids.index_pt==0]) & set(pts_segids.index_ln[pts_segids.index_pt==1])
    t2 = set(pts_segids.index_ln[pts_segids.index_pt==2]) & set(pts_segids.index_ln[pts_segids.index_pt==3])
    return pd.Series({'start':t1,'end':t2})


def call_apply_fn(df):
    return df.apply(apply_fn)

pool = mproc.Pool(mproc.cpu_count()) 
results = pool.map(call_apply_fn, np.array_split(new.geometry.ix[0:10],mproc.cpu_count()))
temp = pd.concat(results)
temp
# apply_fn(new.geometry[146])


In [ ]:
upoi = pd.read_pickle('Updated_POI.pkl')
upoi.head().T

In [ ]:
t = np.array([len(row['end'])+len(row['start']) for idx,row in upoi.iterrows()])
len(upoi.ix[np.where(t>2)])

In [ ]:
np.where(new.geom_equals(purp.geometry[5]))

In [ ]:
temp = upoi['start'].apply(lambda x: poi.ix[list(x)].mean(axis=0).round())
temp.fillna(0,inplace=True)
temp.drop('total', axis=1)
feat1 = temp.as_matrix()

In [ ]:
temp = upoi['start'].apply(lambda x: poi.ix[list(x)].mean(axis=0).round())
temp.fillna(0,inplace=True)
temp.drop('total', axis=1, inplace=True)
feat1 = temp.as_matrix()

temp = upoi['end'].apply(lambda x: poi.ix[list(x)].mean(axis=0).round())
temp.fillna(0,inplace=True)
temp.drop('total', axis=1, inplace=True)
feat2 = temp.as_matrix()

# temp = pd.concat([new.Start.dt.hour, new.Start.dt.month, new.Start.dt.dayofweek],axis=1)
# feat3 = temp.as_matrix()
    
feat1.shape, feat2.shape

In [ ]:
feat1 = feat1[purp['seg']]
feat2 = feat2[purp['seg']]
data = pd.concat([purp.Start.dt.hour,purp.Start.dt.month,purp.Start.dt.dayofweek],axis=1)
feat3 = data.as_matrix()

In [ ]:
data = pd.concat([purp.Purpose,purp.Start.dt.hour],axis=1)
sns.boxplot(x="Purpose",y="Start",data=data)
plt.title('Hour',fontsize=12)

In [ ]:
labels = list(calendar.month_abbr)
data = pd.concat([purp.Purpose,purp.Start.dt.month],axis=1)
sns.boxplot(x="Purpose",y="Start",data=data)
plt.yticks(plt.yticks()[0],labels)
plt.show()

In [ ]:
labels = list(calendar.day_abbr)
data = pd.concat([purp.Start.dt.dayofweek,purp.Purpose],axis=1)
sns.boxplot(x="Purpose",y="Start",data=data)
plt.yticks(plt.yticks()[0],labels)
plt.show()

In [ ]:
def apply_fn(row):
        
    t = list(row.geometry.coords)
    lonlats = snap(row)
    pts = [Point(lonlat) for lonlat in [lonlats[0],lonlats[-1]]]
    pts_gpdf = gpd.GeoDataFrame(pts, columns=['geometry'])
    pts_segs_idx, pts_no_segs = pts2segs(pts_gpdf, ph_str_gpd, bfr_crs=3559, init_crs=4326, close_jn_dist=10, far_jn_dist=30)

    pts_segids = pts_segs_idx.merge(ph_str_gpd[['SEG_ID']], left_on=['index_ln'], right_index=True)
    pts_segids.sort_values(by=['index_pt'],inplace=True)
    pts_segids.reset_index(inplace=True,drop=True)

    t1 = set(pts_segids.index_ln[pts_segids.index_pt==0]) & segments.segments[row.name]
    t2 = set(pts_segids.index_ln[pts_segids.index_pt==1]) & segments.segments[row.name]
    return pd.Series({'start':t1,'end':t2})
    
def call_apply_fn(df):
    return df.apply(apply_fn,axis=1)

pool = mproc.Pool(mproc.cpu_count()) 
results = pool.map(call_apply_fn, np.array_split(new.ix[0:10],mproc.cpu_count()))
temp = pd.concat(results)
# temp.to_pickle('Updated_POI.pkl')

In [ ]:
from pandas.tools.plotting import radviz
radviz(data,'Label')

In [ ]:
# X = np.concatenate((feat1,feat2,feat3),axis=1)
# np.save('Training_Feature.npy',X)
np.linspace(10, 100, num=10,dtype=int)

### SVM classifier with just time as feature

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

data = pd.concat([purp.Start.dt.hour,purp.Start.dt.month,purp.Start.dt.dayofweek,purp.Purpose],axis=1)
data.columns = ['Hour','Month','Day','Label']
data.head()

data['Label'] = data['Label'].astype('category')
labels = dict( enumerate(data.Label.cat.categories) )
data['Label'] = data['Label'].cat.codes

X = np.array(data.ix[:, data.columns != 'Label'])
y = np.array(data.Label)
X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)
    
clf = SVC(kernel='rbf', gamma = 0.9, C=10)
clf.fit(X_train, y_train)

y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))

### SVM classifier with POI and Time as classifier

In [ ]:
f1 = ["start"+"_"+col for col in poi.columns.tolist()[:-1]]
f2 = ["end"+"_"+col for col in poi.columns.tolist()[:-1]]
cols = f1+f2+['month','time','day']
cols

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

X = np.load('Training_Feature.npy')
temp = purp.Purpose.astype('category')
labels = dict( enumerate(temp.cat.categories))
y = np.array(temp.cat.codes)
X = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3)
    
clf = SVC(kernel='rbf', gamma = 0.05, C=10)
clf.fit(X_train, y_train)

y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred, target_names=list(labels.values())))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

X = np.load('Training_Feature.npy')
temp = purp.Purpose.astype('category')
labels = dict( enumerate(temp.cat.categories) )
y = np.array(temp.cat.codes)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3)

clf = SVC(kernel='rbf', gamma = 0.01, C=10)
clf.fit(X_train, y_train)

y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred, target_names=list(labels.values())))


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
# X = np.load('Training_Feature.npy')
X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=0)

clf = AdaBoostClassifier(n_estimators=50)
clf.fit(X_train, y_train)

y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred, target_names=list(labels.values())))

sorted(list(zip(cols,clf.feature_importances_)),key = lambda w: w[1],reverse=True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

X = np.load('Training_Feature.npy')
X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=0)

clf = RandomForestClassifier(n_estimators=500)
clf.fit(X_train, y_train)

y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred, target_names=list(labels.values())))
# sorted(list(zip(cols,clf.feature_importances_)),key = lambda w: w[1],reverse=True)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true,y_pred)

In [ ]:
Counter(purp.Purpose)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=0)

clf = GradientBoostingClassifier(n_estimators=500,max_depth=10)
clf.fit(X_train, y_train)

y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred, target_names=list(labels.values())))
# sorted(list(zip(cols,clf.feature_importances_)),key = lambda w: w[1],reverse=True)
confusion_matrix(y_true,y_pred)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler


from __future__ import print_function

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-2, 2e-3],
                     'C': [10, 100, 1000]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(C=10), tuned_parameters, cv=5,
                       scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

### Segment Feature Creation

In [ ]:
crash = pickle.load(open('data/Collision_Stats.pkl','rb'))
crime = pickle.load(open('data/Crime_Stats.pkl','rb'))
park = pickle.load(open('data/Park_Stats.pkl','rb'))
three11 = pickle.load(open('data/311_Requested_Date_Stats.pkl','rb'))
bike = pd.read_csv('../phillydata/Philly_Bikelanes.csv',index_col=0)
slope = pd.read_csv('../phillydata/Slope_Segment.csv',index_col=0)


traces = pd.read_pickle('Segments.pkl').segments
traces = traces.apply(lambda x: [ph_str_gpd.SEG_ID[t] for t in x])
segments = set().union(*traces)

inpt = ['crash','crime','park','three11']
final = []
for i in inpt:
    data = eval(i)
    temp = list(zip(*data.keys()))
    temp = pd.DataFrame(list(zip(temp[0],temp[1],data.values())),columns=['seg','month','value'])
    temp = temp.pivot(index='seg',columns='month',values='value')
    temp = temp[temp.index.isin(segments)]
    temp = temp[temp.columns[(temp.columns>='2014-05') & (temp.columns<='2016-04')]].to_sparse()
    missing = list(segments-set(temp.index))
    temp = temp.loc[temp.index.tolist() + missing]
    temp.columns = [i+"_"+col for col in temp.columns]
    final.append(temp)
feature = pd.concat(final,axis=1)
feature['bike_lane'] = bike.loc[feature.index.tolist()].BIKELANE
feature['slope'] = slope.loc[feature.index.tolist()].ELEV_SLOPE
feature = feature.to_sparse()


In [ ]:
feature.to_pickle('Complete_Training_Feature.pkl')

In [ ]:
feat = pd.read_pickle('Complete_Training_Feature.pkl')

In [6]:
from scipy.sparse import csr_matrix,vstack
from sklearn.decomposition import TruncatedSVD

feature = pd.read_pickle('Complete_Training_Feature.pkl')
X = np.load('Training_Feature.npy')
traces = pd.read_pickle('Segments.pkl').segments
traces = traces.apply(lambda x: [ph_str_gpd.SEG_ID[t] for t in x])
inpt = ['crash','crime','park','three11']

flag = True
for idx,row in purp.ix[:99].iterrows():
    f = [i+"_"+row.Start.strftime("%Y-%m") for i in inpt]
    f = f + ['bike_lane','slope']

    t = feature.loc[traces[row.seg],f]
    t = t.reindex(index=feature.index,columns=feature.columns)
    t.fillna(0,inplace=True)
    
    temp = csr_matrix(np.append(t.values.ravel(),X[idx]))
    
    if(flag): 
        flag = False
        feat = temp
    else: feat = vstack([feat,temp])
    

In [ ]:
MaxAbsScaler().fit_transform(feature.slope.reshape(-1,1))

In [ ]:
MaxAbsScaler().fit_transform(feat)

In [ ]:
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=50)
svd.fit(feat) 


print(svd.explained_variance_ratio_) 

print(svd.explained_variance_ratio_.sum()) 


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis


names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", 
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(kernel='rbf',gamma=0.01, C=10),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(n_estimators=100),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

X = np.load('Training_Feature.npy')
temp = purp.Purpose.astype('category')
labels = dict( enumerate(temp.cat.categories))
y = np.array(temp.cat.codes)

X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)
for name, clf in zip(names, classifiers):
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print(name,score)


In [ ]:
from scipy.sparse import *

In [ ]:
temp = load_npz('Sparse_Matrix.npz')

In [8]:
f = open('Final_Report.txt','w')

test = [10,20,30,40]

for val in test: 
    svd = TruncatedSVD(n_components=val)
    feat2 = svd.fit_transform(feat)
    if(val==40):
        print(8/0)
    name = 'Train_'+str(val)+'.npy'
    np.save(name,feat2)
    print("Dimension: "+str(val)+" Explained variance ratio: "+str(svd.explained_variance_ratio_.sum())+"\n",file=f)

f.close()

ZeroDivisionError: division by zero